# GOLD PRICE MOVEMENT PREDICTION

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from scipy.stats import uniform
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, log_loss, accuracy_score

# Initialize StandardScaler
scaler = StandardScaler()

In [7]:
gold_sentiment = pd.read_csv('result_Data\gold_sentiment.csv')
gold_financial = pd.read_csv('result_Data\combined_dataset.csv')
mixed_data = pd.read_csv('result_Data\mixed_dataset.csv')

In [8]:
gold_sentiment.head()

,date,sentiment_score,gold_price,gold_price_after_1_day
0,2015-01-02,-0.4019,1186.2,1204.0
1,2015-01-05,0.4404,1204.0,1219.4
2,2015-01-07,-0.5106,1210.7,1208.5
3,2015-01-09,0.0000,1216.1,1232.8
4,2015-01-12,0.0000,1232.8,1234.4


In [9]:
gold_financial.head()

,date,gold_price,usd_rate,oil_price,bond_yield_price,interest_rate,inflation_rate,gold_price_after_1_day
0,2015-01-02,1186.2,91.08,52.69,2.114,0.12,1.71,1204.0
1,2015-01-05,1204.0,91.38,50.04,2.034,0.12,1.64,1219.4
2,2015-01-06,1219.4,91.50,47.93,1.938,0.12,1.56,1210.7
3,2015-01-07,1210.7,91.89,48.65,1.969,0.12,1.57,1208.5
4,2015-01-08,1208.5,92.37,48.79,2.018,0.12,1.62,1216.1


In [10]:
mixed_data.head()

,date,sentiment_score,usd_rate,oil_price,bond_yield_price,interest_rate,inflation_rate,gold_price,gold_price_after_1_day
0,2015-01-02,-0.4019,91.08,52.69,2.114,0.12,1.71,1186.2,1204.0
1,2015-01-05,0.4404,91.38,50.04,2.034,0.12,1.64,1204.0,1219.4
2,2015-01-07,-0.5106,91.89,48.65,1.969,0.12,1.57,1210.7,1208.5
3,2015-01-09,0.0000,91.94,48.36,1.950,0.12,1.62,1216.1,1232.8
4,2015-01-12,0.0000,91.98,46.07,1.909,0.12,1.57,1232.8,1234.4


## CLASSIFICATION MODEL -XGBoost Classification

## Predicting Gold Price Movement with Sentiment Data

In [13]:
gold_sentiment.head()

,date,sentiment_score,gold_price,gold_price_after_1_day
0,2015-01-02,-0.4019,1186.2,1204.0
1,2015-01-05,0.4404,1204.0,1219.4
2,2015-01-07,-0.5106,1210.7,1208.5
3,2015-01-09,0.0000,1216.1,1232.8
4,2015-01-12,0.0000,1232.8,1234.4


In [14]:
gold_sentiment.shape

(1209, 4)

In [15]:
gold_sentiment['price_change_pct'] = ((gold_sentiment['gold_price'] - gold_sentiment['gold_price_after_1_day']) / gold_sentiment['gold_price_after_1_day']) * 100


In [16]:
print(gold_sentiment['price_change_pct'].max())
print(gold_sentiment['price_change_pct'].min())

3.2661264995917345
-4.484271022383559


In [17]:

# Create a function to create the target colunm with the price change categorized to Buy, Sell, and Hold
# Define thresholds for classification
buy_threshold = 0.1  # Adjust as needed
sell_threshold = -0.1
# Apply categorization based on price change
def classify_price_movement(change):
    if change > buy_threshold:
        return "Buy"
    elif change < sell_threshold:
        return "Sell"
    else:
        return "Hold"


In [18]:
# Create the target column with classifications
gold_sentiment['target'] = gold_sentiment['price_change_pct'].apply(classify_price_movement)
gold_sentiment.head()

,date,sentiment_score,gold_price,gold_price_after_1_day,price_change_pct,target
0,2015-01-02,-0.4019,1186.2,1204.0,-1.478405,Sell
1,2015-01-05,0.4404,1204.0,1219.4,-1.262916,Sell
2,2015-01-07,-0.5106,1210.7,1208.5,0.182044,Buy
3,2015-01-09,0.0000,1216.1,1232.8,-1.354640,Sell
4,2015-01-12,0.0000,1232.8,1234.4,-0.129618,Sell


In [19]:

X_sent = gold_sentiment.drop(columns=['date','gold_price_after_1_day','price_change_pct','target'])
y_sent = gold_sentiment["target"].map({"Buy": 0, "Hold": 1, "Sell": 2})  # Map categories to integers


In [20]:
y_sent.value_counts()

target
2    563
0    500
1    146
Name: count, dtype: int64

In [21]:
# split the dataset to train and test set
X_train_sent, X_test_sent, y_train_sent, y_test_sent = train_test_split(X_sent, y_sent, test_size=0.2, random_state=42)

In [22]:
# Fit and transform the scaler on the training data
X_train_scaled = scaler.fit_transform(X_train_sent)
#  transform the test data 
X_test_scaled = scaler.transform(X_test_sent)

In [23]:
model_sent = xgb.XGBClassifier(objective='multi:softmax', num_class=3, learning_rate=0.1, max_depth=5, n_estimators=100)
model_sent.fit(X_train_scaled, y_train_sent)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [24]:
y_pred_sent = model_sent.predict(X_test_scaled)


In [25]:
# Classification Report
print(classification_report(y_test_sent, y_pred_sent, target_names=["Buy", "Hold", "Sell"]))

# Calculate AUC
y_pred_proba = model_sent.predict_proba(X_test_scaled)

# Macro-average AUC
auc_score_macro = roc_auc_score(y_test_sent, y_pred_proba, multi_class='ovr', average='macro')
print(f'Macro-Average AUC Score: {auc_score_macro}')

# Weighted-average AUC
auc_score_weighted = roc_auc_score(y_test_sent, y_pred_proba, multi_class='ovr', average='weighted')
print(f'Weighted-Average AUC Score: {auc_score_weighted}')

# Accuracy
accuracy = accuracy_score(y_test_sent, y_pred_sent)
print("Accuracy:", accuracy)

# F1 Score 
f1 = f1_score(y_test_sent, y_pred_sent, average='weighted')  
print("F1 Score:", f1)

# Multi-class Log Loss
m_logloss = log_loss(y_test_sent, y_pred_proba)
print("Multi-class Log Loss:", m_logloss)



              precision    recall  f1-score   support

         Buy       0.41      0.41      0.41       100
        Hold       0.20      0.04      0.06        26
        Sell       0.49      0.57      0.52       116

    accuracy                           0.45       242
   macro avg       0.36      0.34      0.33       242
weighted avg       0.42      0.45      0.43       242

Macro-Average AUC Score: 0.49685223450603083
Weighted-Average AUC Score: 0.48588519947354775
Accuracy: 0.4462809917355372
F1 Score: 0.42659233064975616
Multi-class Log Loss: 1.071334508147614


### Hyperparameter Tunning on the model model_sent 

In [27]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss

# defining parameter grid
param_grid = {
    'n_estimators': [200, 400, 600],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.2, 0.1, 0.01],
    'subsample': [0.6,0.8, 1.0],
    'colsample_bytree': [0.6,0.8, 1.0],
    'gamma' : [0.5,1,1.5,1,5]
    
}

# Initialize the model
xgb_clf_sent = XGBClassifier(eval_metric='mlogloss')  # for multiclass

# Set up GridSearchCV
gs_sent = GridSearchCV(estimator=xgb_clf_sent, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# Fit the model to the training data
gs_sent.fit(X_train_scaled, y_train_sent)

# Predict probabilities for AUC calculation
y_pred_proba_sent = gs_sent.predict_proba(X_test_scaled)

# Macro-average AUC
auc_score_macro = roc_auc_score(y_test_sent, y_pred_proba_sent, multi_class='ovr', average='macro')
print(f'Macro-Average AUC Score: {auc_score_macro}')

# Weighted-average AUC
auc_score_weighted = roc_auc_score(y_test_sent, y_pred_proba_sent, multi_class='ovr', average='weighted')
print(f'Weighted-Average AUC Score: {auc_score_weighted}')

# Accuracy
y_pred = gs_sent.predict(X_test_scaled)
accuracy = accuracy_score(y_test_sent, y_pred)
print("Accuracy:", accuracy)

# F1 Score (you can specify 'average' depending on your classification type)
f1 = f1_score(y_test_sent, y_pred, average='weighted')  # or 'macro', 'micro', depending on needs
print("F1 Score:", f1)

# Multi-class Log Loss
m_logloss = log_loss(y_test_sent, y_pred_proba_sent)
print("Multi-class Log Loss:", m_logloss)

# Best parameters from GridSearchCV
print("Best Parameters:", gs_sent.best_params_)
print("Best Accuracy from CV:", gs_sent.best_score_)


Fitting 3 folds for each of 1215 candidates, totalling 3645 fits
Macro-Average AUC Score: 0.5649882092107282
Weighted-Average AUC Score: 0.5600976920599201
Accuracy: 0.4834710743801653
F1 Score: 0.40425113277658103
Multi-class Log Loss: 0.9537816808736028
Best Parameters: {'colsample_bytree': 1.0, 'gamma': 5, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 600, 'subsample': 0.6}
Best Accuracy from CV: 0.4767257017223366


# Predicting Gold Price Movement with Financial Data

In [29]:
gold_financial.head()

,date,gold_price,usd_rate,oil_price,bond_yield_price,interest_rate,inflation_rate,gold_price_after_1_day
0,2015-01-02,1186.2,91.08,52.69,2.114,0.12,1.71,1204.0
1,2015-01-05,1204.0,91.38,50.04,2.034,0.12,1.64,1219.4
2,2015-01-06,1219.4,91.50,47.93,1.938,0.12,1.56,1210.7
3,2015-01-07,1210.7,91.89,48.65,1.969,0.12,1.57,1208.5
4,2015-01-08,1208.5,92.37,48.79,2.018,0.12,1.62,1216.1


In [30]:
gold_financial.shape

(2414, 8)

In [31]:
gold_financial['price_change_pct'] = ((gold_financial['gold_price'] - gold_financial['gold_price_after_1_day']) / gold_financial['gold_price_after_1_day']) * 100


In [32]:
# Create the target column with classifications
gold_financial['target'] = gold_financial['price_change_pct'].apply(classify_price_movement)
gold_financial.head()

,date,gold_price,usd_rate,oil_price,bond_yield_price,interest_rate,inflation_rate,gold_price_after_1_day,price_change_pct,target
0,2015-01-02,1186.2,91.08,52.69,2.114,0.12,1.71,1204.0,-1.478405,Sell
1,2015-01-05,1204.0,91.38,50.04,2.034,0.12,1.64,1219.4,-1.262916,Sell
2,2015-01-06,1219.4,91.50,47.93,1.938,0.12,1.56,1210.7,0.718593,Buy
3,2015-01-07,1210.7,91.89,48.65,1.969,0.12,1.57,1208.5,0.182044,Buy
4,2015-01-08,1208.5,92.37,48.79,2.018,0.12,1.62,1216.1,-0.624949,Sell


In [33]:
X_fin = gold_financial.drop(columns=['date','gold_price_after_1_day','price_change_pct','target'])
y_fin = gold_financial["target"].map({"Buy": 0, "Hold": 1, "Sell": 2})  # Map categories to integers


In [34]:
# split the dataset to train and test set
X_train_fin, X_test_fin, y_train_fin, y_test_fin = train_test_split(X_fin, y_fin, test_size=0.2, random_state=42)

# Fit and transform the scaler on the training data
X_train_fin_scaled = scaler.fit_transform(X_train_fin)
#  transform the test data 
X_test_fin_scaled = scaler.transform(X_test_fin)


In [35]:
model_fin = xgb.XGBClassifier(objective='multi:softmax', num_class=3, learning_rate=0.1, max_depth=5, n_estimators=100)
model_fin.fit(X_train_fin_scaled, y_train_fin)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [36]:
y_pred_fin = model_fin.predict(X_test_fin_scaled)

In [37]:
X_test_fin_scaled.shape

(483, 6)

In [38]:
# Classification Report
print(classification_report(y_test_fin, y_pred_fin, target_names=["Buy", "Hold", "Sell"]))

# Calculate AUC
y_pred_proba_fin = model_fin.predict_proba(X_test_fin_scaled)

# Macro-average AUC
auc_score_macro_ = roc_auc_score(y_test_fin, y_pred_proba_fin, multi_class='ovr', average='macro')
print(f'Macro-Average AUC Score: {auc_score_macro_}')

# Weighted-average AUC
auc_score_weighted = roc_auc_score(y_test_fin, y_pred_proba_fin, multi_class='ovr', average='weighted')
print(f'Weighted-Average AUC Score: {auc_score_weighted}')

# Accuracy
accuracy = accuracy_score(y_test_fin, y_pred_fin)
print("Accuracy:", accuracy)

# F1 Score 
f1 = f1_score(y_test_fin, y_pred_fin, average='weighted')  
print("F1 Score:", f1)

# Multi-class Log Loss
m_logloss = log_loss(y_test_fin, y_pred_proba_fin)
print("Multi-class Log Loss:", m_logloss)

              precision    recall  f1-score   support

         Buy       0.39      0.46      0.42       188
        Hold       0.00      0.00      0.00        62
        Sell       0.47      0.52      0.49       233

    accuracy                           0.43       483
   macro avg       0.29      0.33      0.30       483
weighted avg       0.38      0.43      0.40       483

Macro-Average AUC Score: 0.5217345619467406
Weighted-Average AUC Score: 0.5093564359860525
Accuracy: 0.42857142857142855
F1 Score: 0.40098423648086406
Multi-class Log Loss: 1.0315448040485875


### Hyperparameter Tunning on the model model_fin

In [40]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss

# defining parameter grid

param_grid = {
    'n_estimators': [200, 400, 600],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.2, 0.1, 0.01],
    'subsample': [0.6,0.8, 1.0],
    'colsample_bytree': [0.6,0.8, 1.0],
    'gamma' : [0.5,1,1.5,1,5]   
}
# Initialize the model
xgb_clf_fin = XGBClassifier(eval_metric='mlogloss')  # for multiclass

# Set up GridSearchCV
gs_fin = GridSearchCV(estimator=xgb_clf_fin, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# Fit the model to the training data
gs_fin.fit(X_train_fin_scaled, y_train_fin)

# Predict probabilities for AUC calculation
y_pred_proba_fin = gs_fin.predict_proba(X_test_fin_scaled)

# Macro-average AUC
auc_score_macro = roc_auc_score(y_test_fin, y_pred_proba_fin, multi_class='ovr', average='macro')
print(f'Macro-Average AUC Score: {auc_score_macro}')

# Weighted-average AUC
auc_score_weighted = roc_auc_score(y_test_fin, y_pred_proba_fin, multi_class='ovr', average='weighted')
print(f'Weighted-Average AUC Score: {auc_score_weighted}')

# Accuracy
y_pred_rs_fin = gs_fin.predict(X_test_fin_scaled)
accuracy = accuracy_score(y_test_fin, y_pred_rs_fin)
print("Accuracy:", accuracy)

# F1 Score (you can specify 'average' depending on your classification type)
f1 = f1_score(y_test_fin, y_pred_rs_fin, average='weighted')  # or 'macro', 'micro', depending on needs
print("F1 Score:", f1)

# Multi-class Log Loss
m_logloss = log_loss(y_test_fin, y_pred_proba_fin)
print("Multi-class Log Loss:", m_logloss)

# Best parameters from GridSearchCV
print("Best Parameters:", gs_fin.best_params_)
print("Best Accuracy from CV:", gs_fin.best_score_)
print("Best Accuracy:", gs_fin.best_score_)


Fitting 3 folds for each of 1215 candidates, totalling 3645 fits
Macro-Average AUC Score: 0.5018089094207171
Weighted-Average AUC Score: 0.4967998188249511
Accuracy: 0.4306418219461698
F1 Score: 0.401878731290496
Multi-class Log Loss: 1.0101952067813518
Best Parameters: {'colsample_bytree': 0.6, 'gamma': 0.5, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 600, 'subsample': 0.6}
Best Accuracy from CV: 0.46918398165947023
Best Accuracy: 0.46918398165947023


# Predicting Gold Price Movement with mixed Data (Gold data + Sentiment data + Financial Data)

In [42]:
mixed_data.head()

,date,sentiment_score,usd_rate,oil_price,bond_yield_price,interest_rate,inflation_rate,gold_price,gold_price_after_1_day
0,2015-01-02,-0.4019,91.08,52.69,2.114,0.12,1.71,1186.2,1204.0
1,2015-01-05,0.4404,91.38,50.04,2.034,0.12,1.64,1204.0,1219.4
2,2015-01-07,-0.5106,91.89,48.65,1.969,0.12,1.57,1210.7,1208.5
3,2015-01-09,0.0000,91.94,48.36,1.950,0.12,1.62,1216.1,1232.8
4,2015-01-12,0.0000,91.98,46.07,1.909,0.12,1.57,1232.8,1234.4


In [43]:
mixed_data.shape

(1173, 9)

In [44]:
mixed_data['price_change'] = mixed_data['gold_price'] - mixed_data['gold_price_after_1_day']
mixed_data.head()

,date,sentiment_score,usd_rate,oil_price,bond_yield_price,interest_rate,inflation_rate,gold_price,gold_price_after_1_day,price_change
0,2015-01-02,-0.4019,91.08,52.69,2.114,0.12,1.71,1186.2,1204.0,-17.8
1,2015-01-05,0.4404,91.38,50.04,2.034,0.12,1.64,1204.0,1219.4,-15.4
2,2015-01-07,-0.5106,91.89,48.65,1.969,0.12,1.57,1210.7,1208.5,2.2
3,2015-01-09,0.0000,91.94,48.36,1.950,0.12,1.62,1216.1,1232.8,-16.7
4,2015-01-12,0.0000,91.98,46.07,1.909,0.12,1.57,1232.8,1234.4,-1.6


In [45]:
# Create the target column with classifications
mixed_data['target'] = mixed_data['price_change'].apply(classify_price_movement)
mixed_data.head()

,date,sentiment_score,usd_rate,oil_price,bond_yield_price,interest_rate,inflation_rate,gold_price,gold_price_after_1_day,price_change,target
0,2015-01-02,-0.4019,91.08,52.69,2.114,0.12,1.71,1186.2,1204.0,-17.8,Sell
1,2015-01-05,0.4404,91.38,50.04,2.034,0.12,1.64,1204.0,1219.4,-15.4,Sell
2,2015-01-07,-0.5106,91.89,48.65,1.969,0.12,1.57,1210.7,1208.5,2.2,Buy
3,2015-01-09,0.0000,91.94,48.36,1.950,0.12,1.62,1216.1,1232.8,-16.7,Sell
4,2015-01-12,0.0000,91.98,46.07,1.909,0.12,1.57,1232.8,1234.4,-1.6,Sell


In [46]:
X_mix = mixed_data.drop(columns=['date','gold_price_after_1_day','price_change','target'])
y_mix = mixed_data["target"].map({"Buy": 0, "Hold": 1, "Sell": 2})  # Map categories to integers


In [47]:
# split the dataset to train and test set
X_train_mix, X_test_mix, y_train_mix, y_test_mix = train_test_split(X_mix, y_mix, test_size=0.2, random_state=42)

In [48]:
# Fit and transform the scaler on the training data
X_train_mix_scaled = scaler.fit_transform(X_train_mix)
#  transform the test data 
X_test_mix_scaled = scaler.transform(X_test_mix)

In [49]:
model_mix = xgb.XGBClassifier(objective='multi:softmax', num_class=3, learning_rate=0.1, max_depth=5, n_estimators=100)
model_mix.fit(X_train_mix_scaled, y_train_mix)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [50]:
y_pred_mix = model_mix.predict(X_test_mix_scaled)

In [51]:
# Classification Report
print(classification_report(y_test_mix, y_pred_mix, target_names=["Buy", "Hold", "Sell"],zero_division=1))

# Calculate AUC
y_pred_mix_proba = model_mix.predict_proba(X_test_mix_scaled)

# Macro-average AUC
auc_score_macro = roc_auc_score(y_test_mix, y_pred_mix_proba, multi_class='ovr', average='macro')
print(f'Macro-Average AUC Score: {auc_score_macro}')

# Weighted-average AUC
auc_score_weighted = roc_auc_score(y_test_mix, y_pred_mix_proba, multi_class='ovr', average='weighted')
print(f'Weighted-Average AUC Score: {auc_score_weighted}')

# Accuracy
accuracy = accuracy_score(y_test_mix, y_pred_mix)
print("Accuracy:", accuracy)

# F1 Score 
f1 = f1_score(y_test_mix, y_pred_mix, average='weighted', zero_division=1)  
print("F1 Score:", f1)

# Multi-class Log Loss
m_logloss = log_loss(y_test_mix, y_pred_mix_proba)
print("Multi-class Log Loss:", m_logloss)


              precision    recall  f1-score   support

         Buy       0.51      0.47      0.49       113
        Hold       1.00      0.00      0.00         4
        Sell       0.52      0.58      0.55       118

    accuracy                           0.52       235
   macro avg       0.68      0.35      0.35       235
weighted avg       0.53      0.52      0.51       235

Macro-Average AUC Score: 0.507106029365777
Weighted-Average AUC Score: 0.5335981655828185
Accuracy: 0.5191489361702127
F1 Score: 0.5131476753349093
Multi-class Log Loss: 0.8583017028709675


### Hyperparameter Tunning on the model model_mix

In [53]:
# defining parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Initialize the model
xgb_clf_mix = XGBClassifier( eval_metric='mlogloss') # for multiclass

# Set up GridSearchCV
gs = GridSearchCV(estimator=xgb_clf_mix, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# Fit the model to the training data
gs.fit(X_train_mix, y_train_mix)

print("Best Parameters:", gs.best_params_)
print("Best Accuracy:", gs.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'subsample': 1.0}
Best Accuracy: 0.5234599000573441


In [54]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss

# defining parameter grid
param_grid = {
    'n_estimators': [200, 400, 600],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.2, 0.1, 0.01],
    'subsample': [0.6,0.8, 1.0],
    'colsample_bytree': [0.6,0.8, 1.0],
    'gamma' : [0.5,1,1.5,1,5]
    
}

# Initialize the model
xgb_clf_rs_mix = XGBClassifier(eval_metric='mlogloss')  # for multiclass

# Set up GridSearchCV
gs_mix = GridSearchCV(estimator=xgb_clf_rs_mix, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# Fit the model to the training data
gs_mix.fit(X_train_mix_scaled, y_train_mix)

# Predict probabilities for AUC calculation
y_pred_proba_rs_mix = gs_mix.predict_proba(X_test_mix_scaled)

# Macro-average AUC
auc_score_macro = roc_auc_score(y_test_mix, y_pred_proba_rs_mix, multi_class='ovr', average='macro')
print(f'Macro-Average AUC Score: {auc_score_macro}')

# Weighted-average AUC
auc_score_weighted = roc_auc_score(y_test_mix, y_pred_proba_rs_mix, multi_class='ovr', average='weighted')
print(f'Weighted-Average AUC Score: {auc_score_weighted}')

# Accuracy
y_pred_rs_mix = gs_mix.predict(X_test_mix_scaled)
accuracy = accuracy_score(y_test_mix, y_pred_rs_mix)
print("Accuracy:", accuracy)

# F1 Score (you can specify 'average' depending on your classification type)
f1 = f1_score(y_test_mix, y_pred_rs_mix, average='weighted')  # or 'macro', 'micro', depending on needs
print("F1 Score:", f1)

# Multi-class Log Loss
m_logloss = log_loss(y_test_mix, y_pred_proba_rs_mix)
print("Multi-class Log Loss:", m_logloss)

# Best parameters from GridSearchCV
print("Best Parameters:", gs_mix.best_params_)
print("Best Accuracy from CV:", gs_mix.best_score_)


Fitting 3 folds for each of 1215 candidates, totalling 3645 fits
Macro-Average AUC Score: 0.4649061793874634
Weighted-Average AUC Score: 0.5268841107348258
Accuracy: 0.5191489361702127
F1 Score: 0.4087288597926895
Multi-class Log Loss: 0.7753714752958176
Best Parameters: {'colsample_bytree': 1.0, 'gamma': 5, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 400, 'subsample': 0.6}
Best Accuracy from CV: 0.5362257994047132
